In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [ ]:
df1 = pd.read_csv("IBUL.csv", parse_dates = True, infer_datetime_format = True)
df2 = pd.read_csv("GIC.csv", parse_dates = True, infer_datetime_format = True)
df3 = pd.read_csv("LIC.csv", parse_dates = True, infer_datetime_format = True)
df4 = pd.read_csv("Muthoot.csv", parse_dates = True, infer_datetime_format = True)
df5 = pd.read_csv("Reliance.csv", parse_dates = True, infer_datetime_format = True)

In [ ]:
df1['Date'] = pd.to_datetime(df1.Date,format='%Y-%m-%d'); df1.index = df1["Date"]
df2['Date'] = pd.to_datetime(df2.Date,format='%Y-%m-%d'); df2.index = df2["Date"]
df3['Date'] = pd.to_datetime(df3.Date,format='%Y-%m-%d'); df3.index = df3["Date"]
df4['Date'] = pd.to_datetime(df4.Date,format='%Y-%m-%d'); df4.index = df4["Date"]
df5['Date'] = pd.to_datetime(df5.Date,format='%Y-%m-%d'); df5.index = df5["Date"]

In [ ]:
fig, ax = plt.subplots(2,3, figsize = (15,15))

ax[0,0].plot(df1["Close"])
ax[0,0].set_xlabel("Dates")
ax[0,0].set_title("IndiaBulls")
ax[0,0].set_ylabel("Closing Stock Prices")
ax[0,0].legend(["Closing Stock Prices"])

ax[0,1].plot(df2["Close"])
ax[0,1].set_xlabel("Dates")
ax[0,1].set_title("GIC")
ax[0,1].set_ylabel("Closing Stock Prices")
ax[0,1].legend(["Closing Stock Prices"])

ax[0,2].plot(df3["Close"])
ax[0,2].set_xlabel("Dates")
ax[0,2].set_title("LIC")
ax[0,2].set_ylabel("Closing Stock Prices")
ax[0,2].legend(["Closing Stock Prices"])

ax[1,0].plot(df4["Close"])
ax[1,0].set_xlabel("Dates")
ax[1,0].set_title("Muthoot")
ax[1,0].set_ylabel("Closing Stock Prices")
ax[1,0].legend(["Closing Stock Prices"])

ax[1,1].plot(df5["Close"])
ax[1,1].set_xlabel("Dates")
ax[1,1].set_title("Reliance")
ax[1,1].set_ylabel("Closing Stock Prices")
ax[1,1].legend(["Closing Stock Prices"])

In [ ]:
def Plot_prediction(df, title = "No Title Given"):
    data = df.sort_index(ascending=True, axis=0)
    new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
    for i in range(0,len(data)):
        new_data['Date'][i] = data['Date'][i]
        new_data['Close'][i] = data['Close'][i]
    
    #setting index
    new_data.index = new_data.Date
    new_data.drop('Date', axis=1, inplace=True)

    #creating train and test sets
    dataset = new_data.values

    train = dataset[0:179,:]
    test = dataset[179:,:]

    #converting dataset into x_train and y_train
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)

    x_train, y_train = [], []
    for i in range(30,len(train)):
        x_train.append(scaled_data[i-30:i,0])
        y_train.append(scaled_data[i,0])
    x_train, y_train = np.array(x_train), np.array(y_train)

    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(units=50, activation = "relu", return_sequences=True, input_shape=(x_train.shape[1],1)))
    model.add(LSTM(units=50, activation = "relu"))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    history = model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=0)

    loss_per_epoch = history.history['loss']
    plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

    #predicting 70 values, using past 30 from the train data
    inputs = new_data[len(new_data) - len(test) - 30:].values
    inputs = inputs.reshape(-1,1)
    inputs  = scaler.transform(inputs)

    X_test = []
    for i in range(30,inputs.shape[0]):
        X_test.append(inputs[i-30:i,0])
    X_test = np.array(X_test)

    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
    closing_price = model.predict(X_test)
    closing_price = scaler.inverse_transform(closing_price)

    rms=np.sqrt(np.mean(np.power((test-closing_price),2)))
    print("RMS =", rms)

    train = new_data[:179]
    test = new_data[179:]
    test['Predictions'] = closing_price

    plt.figure(figsize=(16,16))
    plt.xlabel("Dates")
    plt.ylabel("Closing Stock Prices")
    plt.plot(train['Close'])

    plt.plot(test[['Close','Predictions']])
    plt.legend(["First 9 months","Next 3 months Actual", "Next 3 months Predicted"])
    plt.title(title)
    plt.show()

In [ ]:
Plot_prediction(df1, "India Bulls Prediction")

In [ ]:
Plot_prediction(df2, "GIC Prediction")

In [ ]:
Plot_prediction(df3, "LIC Predcition")`

In [ ]:
Plot_prediction(df4, "Muthoot Prediction")

In [ ]:
Plot_prediction(df5, "Reliance Prediction")